In [1]:
import sys
import os
import numpy as np
from datetime import datetime
import xarray as xr
import iris
import cartopy.crs as ccrs
from iris.analysis.cartography import unrotate_pole, rotate_pole

# Test code for file processing 
Note - I've converted this into a python script to run in command line: 
See: /home/563/slf563/simple_iris_wrap_RNS.sh

In [2]:
date = '20220602T0000Z'
fdir = '/scratch/jk72/slf563/cylc-run/u-dd992/share/cycle/{}/Regn1/resn_1/RAL3P2/um/'.format(date)
hour = '000'
clat = -68.6
clon = 78
target_lat = -68.5762 # For Davis 
target_lon = 77.9696

In [3]:
fname = 'umnsaa_pb{}'.format(hour)
cubes = iris.load(fdir+fname)

In [4]:
cubes

[<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of air_temperature / (K) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of dew_point_temperature / (K) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of relative_humidity / (%) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of specific_humidity / (1) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of surface_air_pressure / (Pa) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of surface_downwelling_longwave_flux_in_air / (W m-2) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of surface_temperature / (K) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of wind_speed / (m s-1) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of x_wind / (m s-1) (time: 36; grid_latitude: 500; grid_longitude: 500)>,
<iris 'Cube' of y_wind / (m s-1) (time: 36; grid_latitude: 501; grid_longitude: 500)>]

In [5]:
def filter_exact_lat_lon(data,target_lat,target_lon,central_lat,central_lon): 
    grid_lon, grid_lat = rotate_pole(np.array(target_lon), np.array(target_lat), central_lon, central_lat+90)
    point_data = data
    if 'grid_latitude' in data.dims: 
        point_data = point_data.sel(grid_latitude = grid_lat, grid_longitude = grid_lon, method='nearest')
    if 'grid_latitude1' in data.dims: 
        point_data = point_data.sel(grid_latitude1 = grid_lat, grid_longitude1 = grid_lon, method='nearest')
    if 'grid_latitude2' in data.dims: 
        point_data = point_data.sel(grid_latitude2 = grid_lat, grid_longitude2 = grid_lon, method='nearest')

    return point_data



In [6]:
dataout = xr.Dataset()
for i,cube in enumerate(cubes[:]): 

    rotated_lons, rotated_lats = cube.coord('grid_longitude').points, cube.coord('grid_latitude').points
    x, y = np.meshgrid(rotated_lons, rotated_lats)
    lons, lats = unrotate_pole(x, y, clon, clat+90)
    
    data = xr.DataArray.from_iris(cube)
    data = data.assign_coords({'latitude':(('grid_latitude','grid_longitude'),lats)})
    data = data.assign_coords({'longitude':(('grid_latitude','grid_longitude'),lons)})
    
    if i > 0 and data.grid_latitude[0].values != dataout.grid_latitude[0].values:
        data = data.rename({'grid_latitude':'grid_latitude1'})
        data = data.rename({'grid_longitude':'grid_longitude1'})
        data = data.rename({'latitude':'latitude1'})
        data = data.rename({'longitude':'longitude1'})

    if 'grid_longitude' in data.dims and i > 0 and data.grid_longitude[0].values != dataout.grid_longitude[0].values:
        data = data.rename({'grid_longitude':'grid_longitude2'})
        data = data.rename({'grid_latitude':'grid_latitude2'})
        data = data.rename({'latitude':'latitude2'})
        data = data.rename({'longitude':'longitude2'})

    if 'level_height' in list(data.coords):    
        if i > 0 and data.level_height[0].values != dataout.level_height[0].values:
            data = data.rename({'level_height':'level_height1'})
            data = data.rename({'model_level_number':'model_level_number1'})
            
    if 'sigma' in list(data.coords):
        if i > 0 and data.sigma[0].values != dataout.sigma[0].values:
            data = data.rename({'sigma':'sigma1'})

    if (data.attrs['STASH'].item == 229) and (data.attrs['STASH'].section == 15): 
        data = data.rename('potential_vorticity')
    if data.attrs['STASH'].item == 293 and data.attrs['STASH'].section == 30:     
        data = data.rename('w_component_of_wind')
    if data.attrs['STASH'].item == 294 and data.attrs['STASH'].section == 30: 
        data = data.rename('temperature')
    if data.attrs['STASH'].item == 295 and data.attrs['STASH'].section == 30:     
        data = data.rename('specific_humidity')
    if data.attrs['STASH'].item == 296 and data.attrs['STASH'].section == 30: 
        data = data.rename('relative_humidity')
    if data.attrs['STASH'].item == 297 and data.attrs['STASH'].section == 30: 
        data = data.rename('geopotential_height')
    if data.attrs['STASH'].item == 304 and data.attrs['STASH'].section == 30: 
        data = data.rename('heavyside_function')  


    if data.attrs['STASH'].item == 230 and data.attrs['STASH'].section == 3: 
        data = data.rename({'height':'height1'})  
    if data.attrs['STASH'].item == 209 and data.attrs['STASH'].section == 3: 
        data = data.rename({'height':'height1'}) 
    if data.attrs['STASH'].item == 210 and data.attrs['STASH'].section == 3: 
        data = data.rename({'height':'height1'}) 


    dataout = dataout.merge(data)
    
dataout = dataout.assign_attrs({'central lat of rotated grid':clat,
                                'central lon of rotated grid':clon,
                                'history':'Data generated and processed by S. Fiddes sonya.fiddes@utas.edu.au {}'.format(datetime.today().date())})

point_data = filter_exact_lat_lon(dataout,target_lat,target_lon,clat,clon)

In [7]:
dataout

<xarray.Dataset>
Dimensions:                                   (time: 36, grid_latitude: 500,
                                               grid_longitude: 500,
                                               grid_latitude2: 500,
                                               grid_longitude2: 500,
                                               grid_latitude1: 501,
                                               grid_longitude1: 500)
Coordinates: (12/17)
  * time                                      (time) datetime64[ns] 2022-06-0...
  * grid_latitude                             (grid_latitude) float32 -4.5 .....
  * grid_longitude                            (grid_longitude) float32 175.5 ...
    forecast_reference_time                   datetime64[ns] 2022-06-02
    forecast_period                           (time) timedelta64[ns] 00:01:14...
    latitude                                  (grid_latitude, grid_longitude) float64 ...
    ...                                        ...
    latitude2                                 (grid_latitude2, grid_longitude2) float64 ...
    longitude2                                (grid_latitude2, grid_longitude2) float64 ...
  * grid_latitude1                            (grid_latitude1) float32 -4.509...
  * grid_longitude1                           (grid_longitude1) float32 175.5...
    latitude1                                 (grid_latitude1, grid_longitude1) float64 ...
    longitude1                                (grid_latitude1, grid_longitude1) float64 ...
Data variables:
    air_pressure_at_sea_level                 (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    air_temperature                           (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    dew_point_temperature                     (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    relative_humidity                         (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    specific_humidity                         (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    surface_air_pressure                      (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    surface_downwelling_longwave_flux_in_air  (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    surface_temperature                       (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    wind_speed                                (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    x_wind                                    (time, grid_latitude2, grid_longitude2) float64 dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    y_wind                                    (time, grid_latitude1, grid_longitude1) float64 dask.array<chunksize=(1, 501, 500), meta=np.ndarray>
Attributes:
    central lat of rotated grid:  -68.6
    central lon of rotated grid:  78
    history:                      Data generated and processed by S. Fiddes s...

In [8]:
point_data

<xarray.Dataset>
Dimensions:                                   (time: 36, grid_latitude: 1,
                                               grid_longitude: 1,
                                               grid_latitude2: 1,
                                               grid_longitude2: 1,
                                               grid_latitude1: 1,
                                               grid_longitude1: 1)
Coordinates: (12/17)
  * time                                      (time) datetime64[ns] 2022-06-0...
  * grid_latitude                             (grid_latitude) float32 0.018
  * grid_longitude                            (grid_longitude) float32 180.0
    forecast_reference_time                   datetime64[ns] 2022-06-02
    forecast_period                           (time) timedelta64[ns] 00:01:14...
    latitude                                  (grid_latitude, grid_longitude) float64 ...
    ...                                        ...
    latitude2                                 (grid_latitude2, grid_longitude2) float64 ...
    longitude2                                (grid_latitude2, grid_longitude2) float64 ...
  * grid_latitude1                            (grid_latitude1) float32 0.027
  * grid_longitude1                           (grid_longitude1) float32 180.0
    latitude1                                 (grid_latitude1, grid_longitude1) float64 ...
    longitude1                                (grid_latitude1, grid_longitude1) float64 ...
Data variables:
    air_pressure_at_sea_level                 (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    air_temperature                           (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    dew_point_temperature                     (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    relative_humidity                         (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    specific_humidity                         (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    surface_air_pressure                      (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    surface_downwelling_longwave_flux_in_air  (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    surface_temperature                       (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    wind_speed                                (time, grid_latitude, grid_longitude) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    x_wind                                    (time, grid_latitude2, grid_longitude2) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    y_wind                                    (time, grid_latitude1, grid_longitude1) float64 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
Attributes:
    central lat of rotated grid:  -68.6
    central lon of rotated grid:  78
    history:                      Data generated and processed by S. Fiddes s...